In [1]:
import numpy as np 
import pandas as pd 
from scipy import stats
import sklearn as sk 
import matplotlib as plot
import requests as req 
import json
import unidecode
from nba_api.stats.endpoints import leagueseasonmatchups

In [2]:
team_map = {"ATL" : "Atlanta Hawks", "BKN" : "Brooklyn Nets", "BOS": "Boston Celtics", "CHA" : "Charlotte Hornets", "CHI" : "Chicago Bulls", "CLE" : "Cleveland Cavaliers", "DAL" : "Dallas Mavericks", "DEN" : "Denver Nuggets", "DET" : "Detroit Pistons", "GSW" : "Golden State Warriors", "HOU" : "Houston Rockets", "IND" : "Indiana Pacers", "LAC" : "Los Angeles Clippers", "LAL" : "Los Angeles Lakers", "MEM" : "Memphis Grizzlies", "MIA" : "Miami Heat", "MIL" : "Milwaukee Bucks", "MIN" : "Minnesota Timberwolves","NOP" : "New Orleans Pelicans", "NYK" : "New York Knicks", "OKC" : "Oklahoma City Thunder", "ORL" : "Orlando Magic", "PHI" : "Philadelphia 76ers", "PHX" : "Phoenix Suns", "POR" : "Portland Trail Blazers", "SAC" : "Sacramento Kings", "SAS" : "San Antonio Spurs", "TOR" : "Toronto Raptors", "UTA" : "Utah Jazz", "WAS" : "Washington Wizards"}

df_team = pd.read_csv('../data/teams_stats/2019-20_nba_teams.csv')
df_player = pd.read_csv('../data/player_stats/2019-20_nba_players_new.csv')

df_playerid = pd.read_csv('../data/nba_api/all_players.csv')
df_playerid = df_playerid[['id', 'full_name']]

def stripPeriods(s):
    return s.replace('.', '')

df_playerid['full_name'] = df_playerid['full_name'].apply(stripPeriods)


In [3]:
df_playerid.head()

,id,full_name
0,76001,Alaa Abdelnaby
1,76002,Zaid Abdul-Aziz
2,76003,Kareem Abdul-Jabbar
3,51,Mahmoud Abdul-Rauf
4,1505,Tariq Abdul-Wahad


In [4]:
df_player.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Steven Adams/adamsst01,C,26,OKC,63,63,1680,283,478,...,0.582,207,376,583,146,51,67,94,122,684
1,2,Bam Adebayo/adebaba01,PF,22,MIA,72,72,2417,440,790,...,0.691,176,559,735,368,82,93,204,182,1146
2,3,LaMarcus Aldridge/aldrila01,C,34,SAS,53,53,1754,391,793,...,0.827,103,289,392,129,36,87,74,128,1001
3,4,Kyle Alexander/alexaky01,C,23,MIA,2,0,13,1,2,...,NaN,2,1,3,0,0,0,1,1,2
4,5,Nickeil Alexander-Walker/alexani01,SG,21,NOP,47,1,591,98,266,...,0.676,9,75,84,89,17,8,54,57,267


In [5]:
def stripAsterisk(s):
    if s[-1] == '*':
        return s[0:-1]
    else:
        return s

df_team['Team'] = df_team['Team'].apply(stripAsterisk)
df_team = df_team.dropna()
df_team.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PTSA
0,1.0,Dallas Mavericks,75,18175,3124,6772,0.461,1136,3095,0.367,...,791,2729,3520,1849,460,363,953,1462,8776,8405
1,2.0,Milwaukee Bucks,73,17595,3160,6638,0.476,1007,2840,0.355,...,691,3083,3774,1889,526,429,1102,1431,8663,7927
2,3.0,Portland Trail Blazers,74,17835,3122,6749,0.463,952,2525,0.377,...,754,2599,3353,1525,465,451,946,1606,8508,8593
3,4.0,Houston Rockets,72,17380,2936,6512,0.451,1126,3261,0.345,...,704,2484,3188,1557,627,371,1057,1566,8482,8269
4,5.0,Los Angeles Clippers,72,17380,2992,6425,0.466,895,2410,0.371,...,767,2664,3431,1708,511,338,1051,1594,8377,7913


In [6]:
def nameOnly(s):
    new_s = s.split('/')
    return new_s[0]

df_player['Player'] = df_player['Player'].apply(nameOnly)
df_player.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Steven Adams,C,26,OKC,63,63,1680,283,478,...,0.582,207,376,583,146,51,67,94,122,684
1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,...,0.691,176,559,735,368,82,93,204,182,1146
2,3,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,...,0.827,103,289,392,129,36,87,74,128,1001
3,4,Kyle Alexander,C,23,MIA,2,0,13,1,2,...,NaN,2,1,3,0,0,0,1,1,2
4,5,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,...,0.676,9,75,84,89,17,8,54,57,267


In [7]:
df_player = df_player.drop_duplicates(subset="Player", keep="last")

df_player['ID'] = 0

for index, row in df_player.iterrows():
    player = unidecode.unidecode(row['Player']).replace('.', '')
    result = df_playerid[df_playerid['full_name'].str.match(player)]
    if not result.empty:
        df_player.at[index,'ID'] = int(result['id'])

print(len(df_player))

df_player = df_player[df_player['ID'] != 0]

print(len(df_player))

df_player.to_csv('suffering.csv')


529
518


In [8]:
# higher ortg is better
def assignORtg(row):
    FGA = row['FGA']
    ORB = row['ORB']
    TOV = row['TOV']
    FTA = row['FTA']

    TotalP = FGA - ORB + TOV + (0.4 * FTA)

    PTS = row['PTS']

    return (PTS/TotalP) * 100

# Lower drtg is better
def assignDRtg(row):
    FGA = row['FGA']
    ORB = row['ORB']
    TOV = row['TOV']
    FTA = row['FTA']

    TotalP = FGA - ORB + TOV + (0.4 * FTA)

    PTSA = row['PTSA']

    return (PTSA/TotalP) * 100

# def assignNRtg(row):
#     return row['ORTG'] - row['DRTG']


df_team['ORTG'] = df_team.apply(assignORtg, axis=1)
df_team['DRTG'] = df_team.apply(assignDRtg, axis=1)
# df_team['NRTG'] = df_team.apply(assignNRtg, axis=1)
df_team.sort_values(by=['ORTG']).head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,PTSA,ORTG,DRTG
28,29.0,Golden State Warriors,65,15725,2510,5730,0.438,678,2032,0.334,...,2785,1663,534,299,969,1304,6912,7478,103.839913,112.343008
25,26.0,New York Knicks,66,15965,2638,5896,0.447,631,1872,0.337,...,3069,1456,504,313,946,1464,6983,7409,104.739763,111.129444
29,30.0,Charlotte Hornets,65,15750,2425,5586,0.434,785,2231,0.352,...,2781,1549,428,268,949,1223,6687,7126,104.772499,111.650790
27,28.0,Chicago Bulls,65,15675,2573,5762,0.447,793,2282,0.348,...,2721,1510,652,265,1005,1417,6945,7145,104.912535,107.933774
22,23.0,Atlanta Hawks,67,16280,2723,6067,0.449,805,2416,0.333,...,2898,1605,523,341,1086,1548,7488,8022,105.192178,112.693864


In [9]:
def assignOffensivePercentile(row):
    ortgs = np.array(df_team['ORTG'])
    return stats.percentileofscore(ortgs, row['ORTG'])

def assignDefensivePercentile(row):
    drtgs = np.array(df_team['DRTG'])
    drtgs *= -1
    
    return stats.percentileofscore(drtgs, row['DRTG'] * -1)

df_team['ORTGP'] = df_team.apply(assignOffensivePercentile, axis=1)
df_team['DRTGP'] = df_team.apply(assignDefensivePercentile, axis=1)

In [10]:
df_team.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,STL,BLK,TOV,PF,PTS,PTSA,ORTG,DRTG,ORTGP,DRTGP
0,1.0,Dallas Mavericks,75,18175,3124,6772,0.461,1136,3095,0.367,...,460,363,953,1462,8776,8405,114.736952,109.886518,100.000000,46.666667
1,2.0,Milwaukee Bucks,73,17595,3160,6638,0.476,1007,2840,0.355,...,526,429,1102,1431,8663,7927,111.507272,102.033724,90.000000,100.000000
2,3.0,Portland Trail Blazers,74,17835,3122,6749,0.463,952,2525,0.377,...,465,451,946,1606,8508,8593,112.038768,113.158103,93.333333,6.666667
3,4.0,Houston Rockets,72,17380,2936,6512,0.451,1126,3261,0.345,...,627,371,1057,1566,8482,8269,111.379573,108.582609,80.000000,56.666667
4,5.0,Los Angeles Clippers,72,17380,2992,6425,0.466,895,2410,0.371,...,511,338,1051,1594,8377,7913,112.192966,105.978625,96.666667,86.666667


In [11]:
teamAndWhatTheyAreBadAt = {}
for i,j in df_team.iterrows():
    team = j['Team']
    if team not in teamAndWhatTheyAreBadAt:
        offPercentile = j['ORTGP']
        defPercentile = j['DRTGP']
        if offPercentile > defPercentile:
            teamAndWhatTheyAreBadAt[team] = 'defense'
        else:
            teamAndWhatTheyAreBadAt[team] = 'offense'

In [12]:
df_player.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ID
0,1,Steven Adams,C,26,OKC,63,63,1680,283,478,...,207,376,583,146,51,67,94,122,684,203500
1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,...,176,559,735,368,82,93,204,182,1146,1628389
2,3,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,...,103,289,392,129,36,87,74,128,1001,200746
3,4,Kyle Alexander,C,23,MIA,2,0,13,1,2,...,2,1,3,0,0,0,1,1,2,1629734
4,5,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,...,9,75,84,89,17,8,54,57,267,1629638


In [13]:
def assignPlayerOffRtg(row):
    FGA = row['FGA']
    ORB = row['ORB']
    TOV = row['TOV']
    FTA = row['FTA']

    TotalP = FGA - ORB + TOV + (0.4 * FTA)
    if TotalP == 0:
        TotalP = -1

    PTS = row['PTS']

    return (PTS/TotalP) * 100

def assignPlayerDefRtg(row):
    STL = row['STL']
    BLK = row['BLK']
    MP = row['MP']

#     dRtg = ((STL * 0.7) + (BLK * 0.3))/(MP)
    dRtg = ((STL * 0.7) + BLK * 0.3)

    return dRtg

In [14]:
df_player['ORtg'] = df_player.apply(assignPlayerOffRtg, axis=1)
df_player['DRtg'] = df_player.apply(assignPlayerDefRtg, axis=1)

In [15]:
df_player.sort_values(by='DRtg', ascending=False)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,TRB,AST,STL,BLK,TOV,PF,PTS,ID,ORtg,DRtg
162,123,Anthony Davis,PF,26,LAL,62,62,2131,551,1096,...,577,200,91,143,154,156,1618,203076,122.761760,106.6
257,198,James Harden,SG,30,HOU,68,68,2483,672,1514,...,446,512,125,60,308,227,2335,201935,112.693050,105.5
549,451,Ben Simmons,PG,23,PHI,57,57,2017,375,647,...,444,455,119,33,200,186,937,1627732,109.821847,93.2
1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,...,735,368,82,93,204,182,1146,1628389,118.046972,85.3
77,64,Mikal Bridges,SF,23,PHO,73,32,2042,247,484,...,292,129,102,46,73,163,667,1628969,123.564283,85.2
290,223,Jrue Holiday,SG,29,NOP,61,61,2117,458,1006,...,293,408,99,48,184,145,1167,201950,98.414572,83.7
443,357,Dejounte Murray,PG,23,SAS,66,58,1687,294,637,...,383,271,111,17,125,143,720,1627749,97.481722,82.8
564,464,Jayson Tatum,PF,21,BOS,66,66,2265,552,1226,...,460,200,93,57,155,136,1547,1628369,107.266676,82.2
373,294,Kawhi Leonard,SF,28,LAC,57,57,1848,532,1133,...,402,280,103,33,149,113,1543,202695,111.103111,82.0
99,82,Jimmy Butler,SF,30,MIA,58,58,1959,345,758,...,386,350,103,32,127,81,1157,202710,116.986855,81.7


In [16]:
replaceDefenseDict = {}
replaceOffenseDict = {}
for i in teamAndWhatTheyAreBadAt:
    teamKey = i
    vals = list(team_map.values())
    valIdx = vals.index(teamKey)
    teamVal = list(team_map.keys())[valIdx]
    badAt = teamAndWhatTheyAreBadAt[i]
    
    df_players_in_team = df_player[df_player['Tm'].str.match(teamVal)]

    if teamAndWhatTheyAreBadAt[i] == 'defense':
        playersInAteam = df_players_in_team.sort_values(by='DRtg', ascending=False)
        if len(playersInAteam) > 0:
            worstDefensivePlayer = playersInAteam.iloc[-1]
            hisName = worstDefensivePlayer['Player']
            hisOrtg = worstDefensivePlayer['ORtg']
            hisDrtg = worstDefensivePlayer['DRtg']

            threshold = 10
            if hisOrtg == 0:
                threshold = 50
            getPossReplacements = df_player[(df_player['ORtg'] <= hisOrtg + threshold) &
                                            (df_player['ORtg'] >= hisOrtg - threshold) &
                                            (df_player['DRtg'] > hisDrtg)]

            possReplNames = getPossReplacements['Player'].tolist()
            if hisName not in replaceDefenseDict:
                replaceDefenseDict[hisName] = possReplNames
    else:
        playersInAteam = df_players_in_team.sort_values(by='ORtg', ascending=False)
        if len(playersInAteam) > 0:
            worstOffensivePlayer = playersInAteam.iloc[-1]
            hisName = worstOffensivePlayer['Player']
            hisOrtg = worstOffensivePlayer['ORtg']
            hisDrtg = worstOffensivePlayer['DRtg']
            threshold = 5
            if hisDrtg == 0:
                threshold = 50
            getPossReplacements = df_player[(df_player['DRtg'] <= hisDrtg + threshold) &
                                            (df_player['DRtg'] >= hisDrtg - threshold) &
                                            (df_player['ORtg'] > hisOrtg)]

            possReplNames = getPossReplacements['Player'].tolist()
            if hisName not in replaceOffenseDict:
                replaceOffenseDict[hisName] = possReplNames

In [17]:
import pickle
f = open("outputreplaceDef.pkl","wb")
pickle.dump(replaceDefenseDict,f)
f.close()
f = open("outputreplaceOff.pkl","wb")
pickle.dump(replaceOffenseDict,f)
f.close()

In [18]:
# -------------------------
#     K-MEANS CLUSTERING
# -------------------------